
### Asserts

Asserts can be globally disabled with an interpreter settings
They shouldn't be used for security checks etc.

---

### Lists

Changes look better in commits with lists in separate lines.
Also put comma behind last item to make it easier to add new items.


This is better

In [ ]:
names = [
    'bob',
    'john',
    'robert',
]

Than this

In [ ]:
names = ['bob','john','robert']

And this

In [ ]:
names = [
    'bob',
    'john',
    'robert'
]

--- 
### Context managers

Context managers are classes with `__enter__` and `__exit__`.

In [27]:

class ContextManager:
    
    def __enter__(self):
        print('enter')
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exit')
     

In [28]:
with ContextManager():
    print('hello')

enter
hello
exit


In [29]:
with ContextManager():
    raise ValueError

enter
exit


ValueError: 

--- 

### Underscores

### `_var` - Single leading underscore  
Private variable. This is just a convention (in PEP8)  
Python will not import import modules with leading underscores automatically.
Unless defined in `__all__`

### `var_` - Single trailing underscore  
If is variable name is already taken, use this.
`set_`,`class_`,`dict_` etc.
This is just a convention.

### `__var` - Double leading underscore  
Used for name mangling.  
Interpreter changes this name to something else, to prevent collision when extended.
 

In [1]:
class TestA:
    def __init__(self):
        self.a = 1
        self._a = 1
        self.__a = 1
        
    def do_something(self):
        print(self.a)
        print(self._a)
        print(self.__a)
        
tsta = TestA()
print(dir(tsta))
tsta.do_something()

# print(tsta.a)
# print(tsta._a)
# print(tsta.__a)
# This explodes -> __a is not here

class TestB(TestA):
    def __init__(self):
        
        super().__init__()
        
        self.a = 3
        self._a = 3
        self.__a = 3

tstb = TestB()
print(dir(tstb))
tstb.do_something()


['_TestA__a', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_a', 'a', 'do_something']
1
1
1
['_TestA__a', '_TestB__a', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_a', 'a', 'do_something']
3
3
1



### `__var__` - Double leading and training underscores - 'Magic' methods 

### `_` - Single underscore - Unused variable

--- 

### String formatting

#### Old string formatting

Isn't deprecated, but should be avoided


In [2]:

name = 'John'
print('Hello %s' % name)


Hello John



#### New string formatting

Better than 'Old string'


In [1]:

name = 'John'
print('Hello {}'.format(name))


Hello John



#### Literal string interpolation

This should be used.
It is usually faster than just `'Hello '+ name`


In [2]:

name = 'John'
print(f'Hello {name}')


Hello John



#### Template strings

Useful when handling 'untrusted' input from users


In [3]:
from string import Template

name = 'John'

tem = Template('Hello $name')
tem.substitute(name=name)


'Hello John'


How untrusted string from user can expose secrets 


In [4]:

class Car:
    
    car_secret_key = 'SECRET'
    
    def __init__(self):
        pass

blue_car = Car()

string_from_evil_user = '{car.car_secret_key}'

print(string_from_evil_user.format(car=blue_car))


SECRET



--- 

### Functions

Functions that can accept other functions are called higher-order functions


In [6]:

def get_car_func(color):
    
    def red(brand):
        return f'Red {brand}'
    
    def blue(brand):
        return f'Blue {brand}'
    
    if color == 'red':
        return red
    else:
        return blue
    
print(get_car_func('red'))

car_fce = get_car_func('red')

print(car_fce('BMW'))


<function get_car_func.<locals>.red at 0x1073dfb70>
Red BMW



Returned function can still access the upper scope.
This is called 'Lexical closure'


In [11]:

def get_car_func(color):
    
    def red(brand):
        return f'RED {color.capitalize()} {brand}'
    
    def other(brand):
        return f'OTHER {color.capitalize()} {brand}'
    
    if color == 'red':
        return red
    else:
        return other


a = get_car_func('red')
print(a('BMW'))

b = get_car_func('blue')
print(b('BMW'))



RED Red BMW
OTHER Blue BMW



### Decorators

Simplest decorator :


In [3]:

def null_decorator(func):
    return func

def say_name():
    return 'John'

name = null_decorator(say_name)

name()


'John'


Method can be permanently decorated with `@`


In [2]:

def null_decorator(func):
    return func

@null_decorator
def say_name():
    return 'John'

say_name()


'John'


More complicated decorator


In [4]:

def title_decorator(func):
    
    def wrapper():
        original_result = func()
        new_result = original_result.title()
        
        return new_result
    
    return wrapper

@title_decorator
def say_name():
    return 'little john'

say_name()

'Little John'


Decorators are applied from bottom to top


In [5]:

def aa_decorator(func):
    
    def wrapper():
        return f'aa_{func()}_aa'
    
    return wrapper


def bb_decorator(func):
    
    def wrapper():
        return f'bb_{func()}_bb'
    
    return wrapper

@bb_decorator
@aa_decorator
def say_name():
    return 'little john'

say_name()


'bb_aa_little john_aa_bb'


This also means that deep levels of decorator stacking weill eventually have an effect on performance

#### kwargs + args


In [8]:

def aa_decorator(func):
    
    def wrapper(*args, **kwargs):
        
        print(f'{func.__name__} called with {args} and {kwargs}')
        
        return f'aa_{func()}_aa'
    
    return wrapper

# This can be useful for debugging
@aa_decorator
def say_name():
    return 'little john'

say_name()
say_name('hello')
say_name('hello', name='bean')


say_name called with () and {}
say_name called with ('hello',) and {}
say_name called with ('hello',) and {'name': 'bean'}


'aa_little john_aa'


Decorating function hides some data of the original function


In [12]:

def hello():
    """This just says hello"""
    
    return "Hello"

print(hello.__name__)
print(hello.__doc__)

decorated_hello = aa_decorator(hello)

print(decorated_hello.__name__)
print(decorated_hello.__doc__)


hello
This just says hello
wrapper
None



Which can be fixed with `functools.wraps`.
You should use that :)


In [20]:

import functools

def aa_decorator(func):
    @functools.wraps(func)    
    def wrapper(*args, **kwargs):
        
        print(f'{func.__name__} called with {args} and {kwargs}')
        
        return f'aa_{func()}_aa'
    
    return wrapper

@aa_decorator
def hello():
    """This just says hello"""
    
    return "Hello"

print(hello.__name__)
print(hello.__doc__)

hello
This just says hello



### *args and **kwargs

This allows functions to accept optional arguments


In [5]:

def some_func(required_param, *args, **kwargs):
    
    print(required_param)
    
    if args:
        print(f'args {args}')
        
    if kwargs:
        print(f'kwargs {kwargs}')


some_func('required')
some_func('required',1,2)
some_func('required',1,2,a='aaa', b='bb')


required
required
args (1, 2)
required
args (1, 2)
kwargs {'a': 'aaa', 'b': 'bb'}



This can be used to pass parameters from one function to another


In [5]:

def some_other_func(required_param, *args, **kwargs):
    
    print(required_param)
    print(args)
    print(kwargs)

def some_func(required_param, *args, **kwargs):
    
    print(required_param)
    print(args)
    print(kwargs)
    
    kwargs['something'] = 'else'
    args += ('aaa', )
    
    some_other_func(required_param, *args, **kwargs)
    
    
some_func('required',1,2,a='aaa', b='bb')


required
(1, 2)
{'a': 'aaa', 'b': 'bb'}
required
(1, 2, 'aaa')
{'a': 'aaa', 'b': 'bb', 'something': 'else'}
red



`Apple` passes all parameters to its parent. 
Therefore if `Fruit` significantly changes, `Apple` will be probably still working.


In [6]:
class Fruit:
    def __init__(self, color, size):
        self.color = color
        self.size = size
        
class Apple(Fruit):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.color = 'red'

apple = Apple('blue', 100)
print(apple.color)




red



### Function argument unpacking

